# Example Model Servers with Seldon

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [16]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [3]:
import json

## Serve SKLearn Iris Model

In order to deploy SKLearn artifacts, we can leverage the [pre-packaged SKLearn inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).


### Default Seldon protocol

To deploy and start serving an SKLearn artifact using Seldon's default protocol, we can use a config like the one below:

In [25]:
%%writefile ../servers/sklearnserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG

Overwriting ../servers/sklearnserver/samples/iris.yaml


We can then apply it to deploy it to our Kubernetes cluster.

In [26]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [27]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed we can send our sklearn model requests

#### REST Requests

In [28]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/sklearn/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}, 'meta': {}}


In [29]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [30]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.6806334106409295
    values: 0.28830360482888695
    values: 0.8975654688578534
    values: 0.24493223527153596
  }
}

Response:
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.14449927217770223, 0.5104835080299303, 0.34501721979236755]}}, 'meta': {}}


#### gRPC Requests

In [31]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.22157381246204433, 0.020827790786912548, 0.7633031884959527, 0.19217598516001178]}}}
Response:
{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.10758420949891327, 0.4796849750770796, 0.4127308154240072]}}}


In [32]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:sklearn -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}}


And delete the model we deployed

In [33]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


### KFServing V2 protocol

We can deploy a SKLearn artifact, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [6]:
%%writefile ./resources/iris-sklearn-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ./resources/iris-sklearn-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [2]:
!kubectl apply -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [3]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [4]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/sklearn/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "5a0bce13-98e8-4588-9fc5-7c8b11140174",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [5]:
!kubectl delete -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In order to deploy XGBoost models, we can leverage the [pre-packaged XGBoost inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).

### Default Seldon protocol

We can deploy a XGBoost model uploaded to an object store by using the XGBoost model server implementation as shown in the config below:

In [7]:
%%writefile resources/iris.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Writing resources/iris.yaml


And then we apply it to deploy it to our kubernetes cluster

In [8]:
!kubectl apply -f resources/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [9]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-default-0-classifier" successfully rolled out


#### Rest Requests

In [10]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/xgboost/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [2.0]}, 'meta': {}}


In [11]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [12]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.617330874549778
    values: 0.4955768380888388
    values: 0.8325074250157055
    values: 0.29623988728724804
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [0.0]}}, 'meta': {}}


#### gRPC Requests

In [13]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.7029137333313852, 0.9422923262038959, 0.7271094013077166, 0.18443703566724434]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [0.0]}}}


In [14]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:xgboost -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [2]}}


And delete the model we deployed

In [15]:
!kubectl delete -f resources/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


### KFServing V2 protocol

We can deploy a XGBoost model, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [29]:
%%writefile ./resources/iris-xgboost-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ../testing/resources/iris-xgboost-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [30]:
!kubectl apply -f ../testing/resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [31]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [34]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/xgboost/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "7b3fc4b1-bba1-4119-b19a-53649bb7b885",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2.0
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [35]:
!kubectl delete -f ../testing/resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the
tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the
TFServing prepackaged serve with the Seldon Protocol, and a second one which
shows how you can deploy it using the tensorlfow protocol (so you can send
requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Seldon protocol.

In [16]:
%%writefile ./resources/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores     
    name: default
    replicas: 1

Writing ./resources/mnist_rest.yaml


In [17]:
!kubectl apply -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [19]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

deployment "tfserving-default-0-mnist-model" successfully rolled out


In [20]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

#### REST Request

In [21]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.5107023057212949
    values: 0.5623608348950615
    values: 0.6111380506359148
    values: 0.8634274436475573
    values: 0.7114740638344804
    values: 0.02366400253262313
    values: 0.05373980946064383
    values: 0.6336957193832369
    values: 0.31838237069205
    values: 0.25523734153796973
    values: 0.6494473416415116
    values: 0.5178738677262822
    values: 0.9978967930437133
    values: 0.9865181169148434
    values: 0.4372797278205718
    values: 0.9999156261160458
    values: 0.8649524372087054
    values: 0.253189612086792
    values: 0.7387007708037302
    values: 0.10057112454365047
    values: 0.4465099774527844
    values: 0.48168923113488804
    values: 0.07453770297813644
    values: 0.8323663319531219
    values: 0.5808100976724994
    values: 0.2626055767227661
    values: 0.4225917458365325
    values: 0.8975728256665774
    values: 0.06087186315458548
    values:

#### gRPC Request

In [22]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 784], 'values': [0.7905009272630374, 0.6899323067703929, 0.9907993896341335, 0.5895573126130481, 0.8422201754088949, 0.6760859565934112, 0.10368434337730892, 0.46328196059353166, 0.2506709392320108, 0.7389318630631916, 0.8401116551519511, 0.5106163021760177, 0.36876078106775634, 0.2615408924878697, 0.9707615391595129, 0.8796529018862914, 0.2331299015118644, 0.5739288446214379, 0.80967100025705, 0.8932752189547999, 0.6091482903772496, 0.0768058232716533, 0.037274095233411964, 0.018561294861397792, 0.73681851661838, 0.6747561699268244, 0.9148477760498436, 0.027060996015030647, 0.8001872111791688, 0.4865357110229812, 0.10865790274610398, 0.6655168488241036, 0.5796011267249119, 0.9112491513159809, 0.036783869439557915, 0.6612610807847592, 0.9127488438631093, 0.3060892921776468, 0.3088611370075066, 0.7149701978885464, 0.2720614878517995, 0.9264828466774991, 0.8166720961676797, 0.04845529063471321, 0.755381821989797

And delete the model we deployed

In [23]:
!kubectl delete -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


### Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Tensorflow protocol.

In [24]:
%%writefile ./resources/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Writing ./resources/halfplustwo_rest.yaml


In [25]:
!kubectl apply -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io/hpt created


In [26]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "hpt-default-0-halfplustwo" rollout to finish: 0 of 1 updated replicas are available...
deployment "hpt-default-0-halfplustwo" successfully rolled out


In [27]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

{'predictions': [2.5, 3.0, 4.5]}


In [28]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:hpt -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

{'outputs': {'x': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '3'}]}, 'floatVal': [2.5, 3, 3.5]}}, 'modelSpec': {'name': 'halfplustwo', 'version': '123', 'signatureName': 'serving_default'}}


In [29]:
!kubectl delete -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io "hpt" deleted


## Serve MLFlow Elasticnet Wines Model
We can deploy an MLFlow model uploaded to an object store by using the MLFlow
model server implementation as the config below:

In [31]:
%%writefile ./resources/elasticnet_wine.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: classifier
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1

Writing ./resources/elasticnet_wine.yaml


In [32]:
!kubectl apply -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [36]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

deployment "mlflow-default-0-classifier" successfully rolled out


#### REST requests

In [37]:
X=!curl -s -d '{"data": {"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
   -X POST http://localhost:8003/seldon/seldon/mlflow/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [5.275558760255382]}, 'meta': {}}


In [38]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [39]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.6222322767587957
    values: 0.39577565785887203
    values: 0.2685673770421858
    values: 0.19681199557736495
    values: 0.900537501354123
    values: 0.10493672254410757
    values: 0.2277284525811598
    values: 0.09690877934175901
    values: 0.11267963142067294
    values: 0.5974612300649484
    values: 0.4557584572877905
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [5.211586546809321]}}, 'meta': {}}


#### gRPC Requests

In [40]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
         -rpc-header seldon:mlflow -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [5.275558760255382]}}


In [41]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 11], 'values': [0.6028454766973937, 0.6516403205650613, 0.9043459156703918, 0.8688722250485928, 0.3441794352036055, 0.5410399292355342, 0.7568715878119939, 0.29364411275420144, 0.09646033584332869, 0.08618732258687511, 0.3550782101827542]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [5.203227537378]}}}


In [42]:
!kubectl delete -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
